**Import the following playlists from the Spotify API:**

-Global Top 50

-USA Top 50

-Canada Top 50

-Australia Top 50

-Ireland Top 50

-UK Top 50

In [1]:
#import the necessary libraries
import numpy as np
import pandas as pd
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#add the playlists from the stored Data file
globalTop = pd.read_csv(r'C:\\Users\\Jessica\\Spot50-Data-Analysis\\Data\\Global-Top-50.csv', 
                        sep = ',', error_bad_lines = False, index_col = False)
usaTop = pd.read_csv(r'C:\\Users\\Jessica\\Spot50-Data-Analysis\\Data\\USA-Top-50.csv', 
                     sep = ',', error_bad_lines = False, index_col = False)
australiaTop = pd.read_csv(r'C:\\Users\\Jessica\\Spot50-Data-Analysis\\Data\\Australia-Top-50.csv', 
                           sep = ',', error_bad_lines = False, index_col = False)
canadaTop = pd.read_csv(r'C:\\Users\\Jessica\\Spot50-Data-Analysis\\Data\\Canada-Top-50.csv', 
                        sep = ',', error_bad_lines = False, index_col = False)
irelandTop = pd.read_csv(r'C:\\Users\\Jessica\\Spot50-Data-Analysis\\Data\\Ireland-Top-50.csv', 
                         sep = ',', error_bad_lines = False, index_col = False)
ukTop = pd.read_csv(r'C:\\Users\\Jessica\\Spot50-Data-Analysis\\Data\\UK-Top-50.csv', 
                    sep = ',', error_bad_lines = False, index_col = False)

In [30]:
#combine the dataframes into one dataframe
allTracks_df = pd.concat([globalTop,usaTop]).drop_duplicates().reset_index(drop=True)
allTracks_df = pd.concat([allTracks_df,australiaTop]).drop_duplicates().reset_index(drop=True)
allTracks_df = pd.concat([allTracks_df,canadaTop]).drop_duplicates().reset_index(drop=True)
allTracks_df = pd.concat([allTracks_df,irelandTop]).drop_duplicates().reset_index(drop=True)
allTracks_df = pd.concat([allTracks_df,ukTop]).drop_duplicates().reset_index(drop=True)
allTracks_df

,Name,Album,Artist,Release Date,Length,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Time Signature
0,STAY (with Justin Bieber),F*CK LOVE 3: OVER YOU,The Kid LAROI,2021-07-23,141805,92,0.03830,0.591,0.764,0.000000,0.1030,-5.484,0.0483,169.928,4
1,INDUSTRY BABY (feat. Jack Harlow),MONTERO,Lil Nas X,2021-09-17,212352,82,0.02210,0.741,0.691,0.000000,0.0476,-7.395,0.0672,150.087,4
2,THATS WHAT I WANT,MONTERO,Lil Nas X,2021-09-17,143901,89,0.00614,0.737,0.846,0.000000,0.0486,-4.510,0.2200,87.981,4
3,Heat Waves,Dreamland (+ Bonus Levels),Glass Animals,2020-08-06,238805,88,0.44000,0.761,0.525,0.000007,0.0921,-6.900,0.0944,80.870,4
4,Bad Habits,Bad Habits,Ed Sheeran,2021-06-25,231041,97,0.04690,0.808,0.897,0.000031,0.3640,-3.712,0.0348,126.026,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,Wasted,Wasted,Digga D,2021-08-13,174545,75,0.17400,0.763,0.666,0.000000,0.1080,-10.844,0.3590,143.000,4
108,Wid It,Green With Envy,Tion Wayne,2021-09-17,181058,56,0.13900,0.768,0.720,0.000000,0.2010,-6.528,0.2300,140.024,4
109,2k17,2k17,Digga D,2021-09-09,206250,69,0.31100,0.836,0.697,0.000000,0.1210,-6.984,0.3690,144.086,4
110,Verdansk,We're All Alone In This Together,Dave,2021-07-22,182285,75,0.28000,0.961,0.502,0.000000,0.1450,-8.605,0.3120,117.078,4


In [38]:
#Add columns to indicate if a track was listed in each of the playlists
allTracks_df['Global'] = allTracks_df['Name'].isin(globalTop['Name'].tolist()).astype(int)
allTracks_df['USA'] = allTracks_df['Name'].isin(usaTop['Name'].tolist()).astype(int)
allTracks_df['Australia'] = allTracks_df['Name'].isin(australiaTop['Name'].tolist()).astype(int)
allTracks_df['Canada'] = allTracks_df['Name'].isin(canadaTop['Name'].tolist()).astype(int)
allTracks_df['Ireland'] = allTracks_df['Name'].isin(irelandTop['Name'].tolist()).astype(int)
allTracks_df['UK'] = allTracks_df['Name'].isin(ukTop['Name'].tolist()).astype(int)
allTracks_df.head()

,Name,Album,Artist,Release Date,Length,Popularity,Acousticness,Danceability,Energy,Instrumentalness,...,Loudness,Speechiness,Tempo,Time Signature,Global,USA,Australia,Canada,Ireland,UK
0,STAY (with Justin Bieber),F*CK LOVE 3: OVER YOU,The Kid LAROI,2021-07-23,141805,92,0.03830,0.591,0.764,0.000000,...,-5.484,0.0483,169.928,4,1,1,1,1,1,1
1,INDUSTRY BABY (feat. Jack Harlow),MONTERO,Lil Nas X,2021-09-17,212352,82,0.02210,0.741,0.691,0.000000,...,-7.395,0.0672,150.087,4,1,1,1,1,1,1
2,THATS WHAT I WANT,MONTERO,Lil Nas X,2021-09-17,143901,89,0.00614,0.737,0.846,0.000000,...,-4.510,0.2200,87.981,4,1,1,1,1,1,1
3,Heat Waves,Dreamland (+ Bonus Levels),Glass Animals,2020-08-06,238805,88,0.44000,0.761,0.525,0.000007,...,-6.900,0.0944,80.870,4,1,1,1,1,1,1
4,Bad Habits,Bad Habits,Ed Sheeran,2021-06-25,231041,97,0.04690,0.808,0.897,0.000031,...,-3.712,0.0348,126.026,4,1,1,1,1,1,1
